In [6]:
import pandas as pd
import sympy as sp
import numpy as np
from scipy.optimize import fsolve
from scipy.stats import gaussian_kde
week = pd.read_csv('df_week_data.csv')
month = pd.read_csv('df_month_data.csv')

In [7]:
ETF_name = week.columns[1:]
#用來排名
Rank_Weekly = pd.DataFrame() 
Rank_Monthly = pd.DataFrame()
#risk-free rate
week_rf = (2.45785 / 100 ) / 52
month_rf = (2.45785 / 100 ) / 12
ETF_name

Index(['XLE', 'AMLP', 'VDE', 'AMJ', 'EMLP', 'XOP', 'MLPI', 'IXC', 'OIH',
       'MLPA', 'IYE', 'MLPX', 'FENY', 'AMZA', 'ERX', 'ATMP', 'IEO', 'TPYP',
       'AMU', 'TAN', 'GUSH', 'GREK', 'XES', 'RYE', 'FXN', 'ICLN', 'IEZ', 'PZD',
       'PBW', 'DIG', 'FCG', 'MLPC', 'QCLN', 'GEX', 'FAN', 'KOL', 'ENFR', 'PXI',
       'ZMLP', 'PBD', 'FILL', 'MLPY', 'DRIP', 'PXE', 'YMLP', 'CRAK', 'IMLP',
       'GASL', 'MLPO', 'FRAK', 'PSCE', 'GRID', 'NLR', 'ERY', 'YMLI', 'PXJ',
       'YLCO', 'PUW', 'DUG', 'ENY', 'MLPG', 'MLPB', 'AMUB', 'GASX', 'CHIE',
       'DDG'],
      dtype='object')

In [16]:
#clear NA to fit pdf
week2 = week
week2 = week2.dropna(
        axis=0,     # 0: 对行进行操作; 1: 对列进行操作
        how='any'   # 'any': 只要存在 NaN 就 drop 掉; 'all': 必须全部是 NaN 才 drop 
        ) 
month2 = month
month2 = month2.dropna(
        axis=0,     # 0: 对行进行操作; 1: 对列进行操作
        how='any'   # 'any': 只要存在 NaN 就 drop 掉; 'all': 必须全部是 NaN 才 drop 
        ) 

# Global Riskiness R

---
### using uniform distribution

In [8]:
def f1(x):
    return sum(np.exp(-1*ret/x)) - len(ret)

In [12]:
result = pd.DataFrame()
for name in ETF_name:
    ret = week[name] - week_rf
    ret = ret * 4000 #調整數量級
    guess = 10**(-5)
    while (guess<1000):
        risk = fsolve(f1,guess)
        if risk != guess:
            break
        guess = guess * 2
        
    ret = month[name] - month_rf
    ret = ret * 1000
    guess2 = 10**(-5)
    while (guess2<1000):
        risk2 = fsolve(f1,guess2)
        if risk2 != guess2:
            break
        guess2 = guess2 * 2

    result[name] = [np.exp(-1 / risk), np.exp(-1 / risk2)]
    #result[name] = [np.exp(-1 / risk2)]
Riskiness = pd.DataFrame()
Riskiness["Riskiness_week_uniform"] = result.T[0] 
Riskiness["Riskiness_month_uniform"] = result.T[1]
Riskiness.head()

C:\Users\smile\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\smile\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


,Riskiness_week_uniform,Riskiness_month_uniform
XLE,[0.4424578176239887],[0.31847578023879]
AMLP,[0.7956913542821963],[0.2613453129655935]
VDE,[0.42313445993495824],[0.2729776183887574]
AMJ,[0.3999712519988899],[0.38548075731649994]
EMLP,[0.6319649981955575],[0.055243570934622614]


In [13]:
#Rank
Rank_Weekly["Riskiness_week_uniform_rank"] = Riskiness.sort_values(by = "Riskiness_week_uniform", ascending= False).index
Rank_Monthly["Riskiness_month_uniform_rank"] = Riskiness.sort_values(by = "Riskiness_month_uniform", ascending= False).index

---
### using fitted pdf

In [17]:
def pdf(data):
    kde = gaussian_kde(data)
    return kde
def f3(x):
    summ = 0 
    for r in ret:
        summ = summ + np.exp(-1*r/x) * pdf(ret).integrate_box_1d(r-10**(-3), r)
    return summ - 1

In [18]:
result2 = pd.DataFrame()
for i in range(0,66):
    ret = week2[ETF_name[i]] - week_rf
    ret = ret * 4000
    guess2 = 10**(-5)
    while (guess2<1000):
        risk2 = fsolve(f3, guess2)
        if risk2 != guess2:
            break
        guess2 = guess2 * 10
    
    ret = month2[ETF_name[i]] - month_rf
    ret = ret * 1000
    guess3 = 10**(-5)
    while (guess3<1000):
        risk3 = fsolve(f3, guess3)
        if risk3 != guess3:
            break
        guess3 = guess3 * 10
    result2[ETF_name[i]] = [np.exp(-1 / risk2), np.exp(-1 / risk3)]
    
Riskiness2 = pd.DataFrame()
Riskiness2["Riskiness_week_fit_pdf"] = result2.T[0] 
Riskiness2["Riskiness_month_fit_pdf"] = result2.T[1]
Riskiness2.head()

C:\Users\smile\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\smile\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


,Riskiness_week_fit_pdf,Riskiness_month_fit_pdf
XLE,[0.7474470653803515],[0.9139988657451691]
AMLP,[0.6282891062810966],[0.9176896014327868]
VDE,[0.7150260638368362],[0.9192702688865181]
AMJ,[0.6760946711891742],[0.9079542283511314]
EMLP,[0.9437877279426673],[0.8602002325586239]


In [19]:
#Rank
Rank_Weekly["Riskiness_week_fitpdf_rank"] = Riskiness2.sort_values(by = "Riskiness_week_fit_pdf", ascending= False).index
Rank_Monthly["Riskiness_month_fitpdf_rank"] = Riskiness2.sort_values(by = "Riskiness_month_fit_pdf", ascending= False).index

# Sharpe Omega measure

In [22]:
def EV(data):
    ave = 0
    tick = (max(data[name])-min(data[name])) / 100
    kde = gaussian_kde(data[name])
    for i in range(0,100):
        x = min(data[name]) + tick * i
        ave = ave + x * kde.integrate_box_1d(x-10**(-3), x)
    return ave
    
def EV_D(data, rf):
    ave = 0
    tick = (max(data[name])-min(data[name])) / 100
    kde = gaussian_kde(data[name])
    for i in range(0,100):
        x = min(data[name]) + tick * i
        if (rf - x) > 0:
            ave = ave + (rf - x) * kde.integrate_box_1d(x-10**(-3), x)
    return ave

In [23]:
expected_value = pd.DataFrame()
expected_value2 = pd.DataFrame()

for name in ETF_name:
    expected_value[name] = [EV(week2), EV(month2), EV_D(week2,week_rf), EV_D(month2, month_rf)]
    #expected_value2[name] = [EV_D(week2,week_rf), EV_D(month2, month_rf)]

EX = expected_value.T
EX.columns = ['EX_all_week','EX_all_month','EX_part_week','EX_part_month']
EX.head()

,EX_all_week,EX_all_month,EX_part_week,EX_part_month
XLE,-0.000288,-0.004168,0.007552,0.010837
AMLP,-0.000199,-0.003690,0.005677,0.011123
VDE,-0.000326,-0.004689,0.007185,0.011096
AMJ,-0.000004,-0.002310,0.006194,0.010780
EMLP,0.001580,0.001555,0.006079,0.007381


In [24]:
SO_weekly = pd.DataFrame()
SO_monthly = pd.DataFrame()
SO_weekly["Weekly Sharpe Omega"] = (EX["EX_all_week"] - week_rf) / EX["EX_part_week"]
SO_monthly["Monthly Sharpe Omega"] = (EX['EX_all_month'] - month_rf) / EX["EX_part_month"]
#排名
Rank_Weekly["Sharpe_Omega_week_rank"] = SO_weekly.sort_values(by = "Weekly Sharpe Omega", ascending= False).index
Rank_Monthly["Sharpe_Omega_month_rank"] = SO_monthly.sort_values(by = "Monthly Sharpe Omega", ascending= False).index

# 用週報酬排名

In [25]:
Rank_Weekly.head(10)

,Riskiness_week_uniform_rank,Riskiness_week_fitpdf_rank,Sharpe_Omega_week_rank
0,ENY,ERY,KOL
1,MLPO,DRIP,NLR
2,PUW,GASL,CRAK
3,ERY,ERX,YLCO
4,GASX,MLPB,FAN
5,GUSH,DIG,ICLN
6,GASL,AMUB,TPYP
7,DRIP,AMZA,CHIE
8,MLPB,MLPG,EMLP
9,AMU,GASX,GEX


# 用月報酬排名

In [26]:
Rank_Monthly.head(10)

,Riskiness_month_uniform_rank,Riskiness_month_fitpdf_rank,Sharpe_Omega_month_rank
0,ENY,GASL,NLR
1,PUW,GUSH,KOL
2,MLPO,ERY,YLCO
3,ERY,IEZ,PZD
4,DRIP,OIH,TPYP
5,GASL,FCG,DDG
6,GUSH,DUG,EMLP
7,GASX,XOP,ICLN
8,ERX,PXE,ATMP
9,XOP,AMZA,DUG


In [28]:
week_rank_B_C = Rank_Weekly
week_rank_B_C.to_csv("rank_with_functionB&C_week.csv")

In [29]:
month_rank_B_C = Rank_Monthly
month_rank_B_C.to_csv("rank_with_functionB&C_month.csv")